In [1]:
# Import packages
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import matplotlib.pyplot as plt

import astropy.units as u
from astropy import wcs
from astropy.io import fits
from astropy.table import Table, Column, hstack, vstack
from astropy.coordinates import SkyCoord

# import kuaizi
from kuaizi.display import display_single, SEG_CMAP
from kuaizi.detection import Data
from kuaizi.utils import padding_PSF

import pickle

In [2]:
# patch = 15
# stack = 'deepCoadd_calexp'
# # stack = 'deepCoadd_calexp_background'
# stack = 'deepCoadd_meas'
# file = f'/projects/MERIAN/repo/DECam/runs/merian/w_2022_02/t9813_deep/20220504T071124Z/{stack}'
# file += f'/9813/{patch}/N708/{stack}_9813_{patch}_N708_hsc_rings_v1_DECam_runs_merian_w_2022_02_t9813_deep_20220504T071124Z.fits'

In [3]:
import lsst.daf.butler as dafButler
butler = dafButler.Butler('/projects/MERIAN/repo')
dataId = dict(tract=9813)
obj = butler.get('objectTable_tract', dataId=dataId, 
                 instrument='DECam', 
                 skymap='hsc_rings_v1', 
                 collections='DECam/runs/merian/w_2022_02/t9813_deep')

In [8]:
obj_cat = Table.from_pandas(obj, index=True)
obj_cat

In [10]:
obj_cat.write('/scratch/gpfs/jiaxuanl/Data/Merian/Catalogs/t9813_deep/merian_objcat.fits')

### Also get RC2 HSC catalog

In [2]:
import lsst.daf.butler as dafButler
butler = dafButler.Butler('/projects/MERIAN/repo')
dataId = dict(tract=9813)
obj = butler.get('objectTable_tract', dataId=dataId, 
                 instrument='HSC', 
                 skymap='hsc_rings_v1', 
                 collections='HSC/runs/RC2/w_2022_04/DM-33402')

In [3]:
obj_cat = Table.from_pandas(obj, index=True)
obj_cat[:5]

objectId,deblend_nChild,detect_isPatchInner,shape_xx,deblend_skipped,yErr,detect_isTractInner,shape_flag,shape_yy,parentObjectId,coord_ra,refExtendedness,detect_isDeblendedSource,footprintArea,detect_isPrimary,detect_isDeblendedModelSource,y,detect_isIsolated,x,merge_peak_sky,sky_object,refFwhm,xErr,detect_fromBlend,xy_flag,shape_xy,coord_dec,refBand,skymap,tract,patch,y_ra,y_decl,y_psfFlux,y_psfFluxErr,y_free_psfFlux,y_free_psfFluxErr,y_free_psfFlux_flag,y_bdE1,y_bdE2,y_bdReB,y_bdReD,y_bdChi2,y_bdFluxB,y_bdFluxBErr,y_bdFluxD,y_bdFluxDErr,y_gaapPsfFlux,y_gaapPsfFluxErr,y_gaap0p5Flux,y_gaap0p5FluxErr,y_gaap0p7Flux,y_gaap0p7FluxErr,y_gaap1p0Flux,y_gaap1p0FluxErr,y_gaap1p5Flux,y_gaap1p5FluxErr,y_gaap2p5Flux,y_gaap2p5FluxErr,y_gaap3p0Flux,y_gaap3p0FluxErr,y_gaapOptimalFlux,y_gaapOptimalFluxErr,y_ixx,y_iyy,y_ixy,y_i_flag,y_ixxPSF,y_iyyPSF,y_ixyPSF,y_iPSF_flag,y_ixxRound,y_iyyRound,y_ixyRound,y_iRound_flag,y_ixxDebiasedPSF,y_iyyDebiasedPSF,y_ixyDebiasedPSF,y_iDebiasedPSF_flag,y_kronRad,y_kronFlux,y_kronFluxErr,y_calibFlux,y_calibFluxErr,y_ap03Flux,y_ap03FluxErr,y_ap03Flux_flag,y_ap06Flux,y_ap06FluxErr,y_ap06Flux_flag,y_ap09Flux,y_ap09FluxErr,y_ap09Flux_flag,y_ap12Flux,y_ap12FluxErr,y_ap12Flux_flag,y_ap17Flux,y_ap17FluxErr,y_ap17Flux_flag,y_ap25Flux,y_ap25FluxErr,y_ap25Flux_flag,y_ap35Flux,y_ap35FluxErr,y_ap35Flux_flag,y_ap50Flux,y_ap50FluxErr,y_ap50Flux_flag,y_ap70Flux,y_ap70FluxErr,y_ap70Flux_flag,y_extendedness,y_blendedness,y_fwhm,y_cModelFlux,y_cModelFluxErr,y_cModelFlux_inner,y_free_cModelFlux,y_free_cModelFluxErr,y_free_cModelFlux_inner,y_free_cModelFlux_flag,y_hsmShapeRegauss_e1,y_hsmShapeRegauss_e2,y_hsmShapeRegauss_sigma,y_hsmShapeRegauss_flag,y_inputCount,y_inputCount_flag,y_inputCount_flag_noInputs,y_psfFlux_area,y_psfFlux_flag,y_psfFlux_flag_apCorr,y_psfFlux_flag_edge,y_psfFlux_flag_noGoodPixels,y_cModel_flag,y_cModel_flag_apCorr,y_gaapFlux_flag,y_gaapFlux_flag_edge,y_gaapFlux_flag_gaussianization,y_gaapOptimalFlux_flag_bigPsf,y_gaap0p5Flux_flag_bigPsf,y_gaap0p7Flux_flag_bigPsf,y_gaap1p0Flux_flag_bigPsf,y_gaap1p5Flux_flag_bigPsf,y_gaap2p5Flux_flag_bigPsf,y_gaap3p0Flux_flag_bigPsf,y_blendedness_flag,y_pixelFlags_bad,y_pixelFlags_clipped,y_pixelFlags_clippedCenter,y_pixelFlags_cr,y_pixelFlags_crCenter,y_pixelFlags_edge,y_pixelFlags_inexact_psf,y_pixelFlags_inexact_psfCenter,y_pixelFlags_interpolated,y_pixelFlags_interpolatedCenter,y_pixelFlags_offimage,y_pixelFlags_saturated,y_pixelFlags_saturatedCenter,y_pixelFlags_sensor_edge,y_pixelFlags_sensor_edgeCenter,y_pixelFlags_suspect,y_pixelFlags_suspectCenter,y_extendedness_flag,y_calib_astrometry_used,y_calib_photometry_reserved,y_calib_photometry_used,y_calib_psf_candidate,y_calib_psf_reserved,y_calib_psf_used,y_apFlux_flag,y_apFlux_flag_apertureTruncated,y_apFlux_flag_sincCoeffsTruncated,y_calibFlux_flag,y_calibFlux_flag_apertureTruncated,y_calibFlux_flag_sincCoeffsTruncated,y_centroid_flag,y_centroid_x,y_centroid_xErr,y_centroid_y,y_centroid_yErr,y_kronFlux_flag,y_kronFlux_flag_bad_radius,y_kronFlux_flag_bad_shape,y_kronFlux_flag_bad_shape_no_psf,y_kronFlux_flag_edge,y_kronFlux_flag_no_fallback_radius,y_kronFlux_flag_no_minimum_radius,y_kronFlux_flag_small_radius,y_kronFlux_flag_used_minimum_radius,y_kronFlux_flag_used_psf_radius,z_ra,z_decl,z_psfFlux,z_psfFluxErr,z_free_psfFlux,z_free_psfFluxErr,z_free_psfFlux_flag,z_bdE1,z_bdE2,z_bdReB,z_bdReD,z_bdChi2,z_bdFluxB,z_bdFluxBErr,z_bdFluxD,z_bdFluxDErr,z_gaapPsfFlux,z_gaapPsfFluxErr,z_gaap0p5Flux,z_gaap0p5FluxErr,z_gaap0p7Flux,z_gaap0p7FluxErr,z_gaap1p0Flux,z_gaap1p0FluxErr,z_gaap1p5Flux,z_gaap1p5FluxErr,z_gaap2p5Flux,z_gaap2p5FluxErr,z_gaap3p0Flux,z_gaap3p0FluxErr,z_gaapOptimalFlux,z_gaapOptimalFluxErr,z_ixx,z_iyy,z_ixy,z_i_flag,z_ixxPSF,z_iyyPSF,z_ixyPSF,z_iPSF_flag,z_ixxRound,z_iyyRound,z_ixyRound,z_iRound_flag,z_ixxDebiasedPSF,z_iyyDebiasedPSF,z_ixyDebiasedPSF,z_iDebiasedPSF_flag,z_kronRad,z_kronFlux,z_kronFluxErr,z_calibFlux,z_calibFluxErr,z_ap03Flux,z_ap03FluxErr,z_ap03Flux_flag,z_ap06Flux,z_ap06FluxErr,z_ap06Flux_flag,

In [4]:
obj_cat.write('/scratch/gpfs/jiaxuanl/Data/Merian/Catalogs/t9813_deep/hsc_objcat.fits')

then cross match using topcat 

### Then match with Song's star catalog on laptop

---
### Test cutout

In [11]:
import lsst.daf.butler as dafButler
import lsst.geom as geom
import lsst.afw.image as afwImage
import lsst.afw.geom as afwGeom
import astropy.units as u

In [4]:
butler = dafButler.Butler('/projects/MERIAN/repo', collections="HSC/runs/RC2/w_2022_04/DM-33402")
skymap = butler.get('skyMap', skymap='hsc_rings_v1')

In [5]:
obj_cat = Table.read('/scratch/gpfs/jiaxuanl/Data/Merian/Catalogs/t9813_deep/COSMOS_lowz_hsc_merian.fits')

In [6]:
import sys
sys.path.append('/home/jiaxuanl/Research/Merian/merian_tractor/scripts/gen_cutout/')

In [22]:
from merian_batch_cutout_hsc import generate_cutout, sky_cone, _get_patches, get_tract_patch_list

In [13]:
radec_list = np.array(
        sky_cone(obj_cat['coord_ra'].data[0], obj_cat['coord_dec'].data[0], 
                 10 * u.Unit('arcsec'), steps=50)).T

In [20]:
coord = geom.SpherePoint(obj_cat['coord_ra'].data[0] * geom.degrees, 
                                  obj_cat['coord_dec'].data[0] * geom.degrees)

In [15]:
img_patches = _get_patches(
        butler, skymap, radec_list, 'g', data_type='deepCoadd_calexp')

"No value in data ID ({'tract': 9813, 'patch': 11, 'band': 'g'}) for required dimension 'skymap'."


In [23]:
patches = get_tract_patch_list(radec_list, skymap)

In [25]:
patches

array([(9813, 11)], dtype=[('tract', '<i8'), ('patch', '<i8')])

In [37]:
data_id = {'tract': 9813, 'patch': 11, 'band': 'g'}

In [38]:
data_id

{'tract': 9813, 'patch': 11, 'band': 'g'}

In [39]:
butler.datasetExists('deepCoadd_calexp', data_id, 
                     collections="HSC/runs/RC2/w_2022_04/DM-33402", skymap='hsc_rings_v1')

True

In [9]:
generate_cutout(butler, skymap, 
                obj_cat['coord_ra'].data[0], obj_cat['coord_dec'].data[0], 
                band='g', data_type='deepCoadd_calexp', verbose=True)

"No value in data ID ({'tract': 9813, 'patch': 11, 'band': 'g'}) for required dimension 'skymap'."
***** No data at 150.58673 1.61481 *****
